In [4]:
# %matplotlib inline
import numpy as np
import k3d
from k3d.colormaps import matplotlib_color_maps
import pathlib
import matplotlib.pyplot as plt
import imageio
import svox2
import torch
import tqdm
import open3d as o3d

INFO - 2022-09-26 09:32:35,365 - rasterization - Cannot import nvdiffrast
/home/tw554/plenoxels/svox2/utils.py:39: UserWarning: CUDA extension svox2.csrc could not be loaded! Operations will be slow.
Please do not import svox in the svox2 source directory.
  warn("CUDA extension svox2.csrc could not be loaded! " +


In [64]:
path = "opt/ckpt/dtu_scan63/reso_256_low_den_lr_2/pts.npy"
pts = np.load(path)

pts.shape

(2644705, 3)

In [57]:
pts.max()

0.8584

In [65]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)

# downsample point cloud
voxel_size = 0.001
downpcd = pcd.voxel_down_sample(voxel_size=voxel_size)
radius_normal = voxel_size * 2

print(np.asarray(downpcd.points).shape)

# downpcd.estimate_normals(
#     search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,max_nn=30)
#     )



(396848, 3)


In [66]:
plt_points = k3d.points(np.asarray(downpcd.points),
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [13]:
downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,max_nn=30)
    )

print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        downpcd, depth=9)

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 89510 / 89439
[Open3D DEBUG] #   Got kernel density: 0.190896 (s), 2447.77 (MB) / 2447.77 (MB) / 2730 (MB)
[Open3D DEBUG] #     Got normal field: 0.624237 (s), 2487.13 (MB) / 2487.13 (MB) / 2730 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.621553e-05 / 5.031852e+00
[Open3D DEBUG] #       Finalized tree: 0.651328 (s), 2491.77 (MB) / 2491.77 (MB) / 2730 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.770962 (s), 2491.77 (MB) / 2491.77 (MB) / 2730 (MB)
[Open3D DEBUG] #Set point constraints: 0.194765 (s), 2491.77 (MB) / 2491.77 (MB) / 2730 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1829780 / 1304160 / 787017
[Open3D DEBUG] Memory Usage: 2491.773 MB
[Open3D DEBUG] # Linear system solved: 1.3348 (s), 2491.77 (MB) / 2491.77 (MB) / 2730 (MB)
[Open3D DEBUG] Got average: 0.0459881 (s), 2491.77 (MB) / 2491.77 (MB) / 2730 (MB)
[Open3D DEBUG] Iso-Value: 4.999108e-01 = 4.474702e+04 / 8.951000e+04
[Open3

In [17]:
np.asarray(mesh.vertices)
np.asarray(mesh.triangles)

array([[    48,     15,     52],
       [    14,     15,     48],
       [    57,     52,     78],
       ...,
       [336919, 336923, 337013],
       [337019, 336925, 337023],
       [336923, 336925, 337019]], dtype=int32)

In [18]:
plt_points = k3d.mesh(np.asarray(mesh.vertices),
                      np.asarray(mesh.triangles),
                      shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot.display()

/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [1]:
import numpy as np
import open3d as o3d
import sklearn.neighbors as skln
from tqdm import tqdm
from scipy.io import loadmat
import multiprocessing as mp
import argparse
import k3d

def sample_single_tri(input_):
    n1, n2, v1, v2, tri_vert = input_
    c = np.mgrid[:n1+1, :n2+1]
    c += 0.5
    c[0] /= max(n1, 1e-7)
    c[1] /= max(n2, 1e-7)
    c = np.transpose(c, (1,2,0))
    k = c[c.sum(axis=-1) < 1]  # m2
    q = v1 * k[:,:1] + v2 * k[:,1:] + tri_vert
    return q

def write_vis_pcd(file, points, colors):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(file, pcd)

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--pts_dir', type=str, default='/home/tw554/plenoxels/opt/ckpt/dtu_scan63/reso_256_low_den_lr_2')
parser.add_argument('--scan', type=int, default=63)
parser.add_argument('--dataset_dir', type=str, default='/home/tw554/plenoxels/data/dtu_eval/SampleSet/MVS Data')
parser.add_argument('--downsample_density', type=float, default=0.2)
parser.add_argument('--patch_size', type=float, default=60)
parser.add_argument('--max_dist', type=float, default=20)
parser.add_argument('--visualize_threshold', type=float, default=10)
args = parser.parse_args(args=[])

thresh = args.downsample_density

pbar = tqdm(total=8)
pbar.set_description('read data pcd')
data_pcd = np.load(f'{args.pts_dir}/pts.npy')

# # rescale pts
# cam = np.load(f'/home/tw554/plenoxels/data/dtu/dtu_scan{args.scan}/cameras_large.npz')
# data_pcd = np.load(f'{args.pts_dir}/pts.npy')
# data_pcd = data_pcd * cam['scale_mat_0'][0,0] + cam['scale_mat_0'][:3,3][None]
# data_pcd = data_pcd[:, :3]

pbar.update(1)
pbar.set_description('random shuffle pcd index')
shuffle_rng = np.random.default_rng()
shuffle_rng.shuffle(data_pcd, axis=0)




masking data pcd:  38%|█████████████████████████████████████▏                                                             | 3/8 [12:52<21:27, 257.46s/it]

random shuffle pcd index:  12%|███████████▌                                                                                | 1/8 [00:00<00:00, 65.86it/s]

In [3]:
pbar.update(1)
pbar.set_description('downsample pcd')
nn_engine = skln.NearestNeighbors(n_neighbors=1, radius=thresh, algorithm='kd_tree', n_jobs=-1)
nn_engine.fit(data_pcd)
rnn_idxs = nn_engine.radius_neighbors(data_pcd, radius=thresh, return_distance=False)
mask = np.ones(data_pcd.shape[0], dtype=np.bool_)
for curr, idxs in enumerate(rnn_idxs):
    if mask[curr]:
        mask[idxs] = 0
        mask[curr] = 1
data_down = data_pcd[mask]

downsample pcd:  25%|█████████████████████████▌                                                                            | 2/8 [00:24<01:13, 12.25s/it]

In [11]:
stl_pcd = o3d.io.read_point_cloud(f'{args.dataset_dir}/Points/stl/stl{args.scan:03}_total.ply')
stl = np.asarray(stl_pcd.points)

plt_points = k3d.points(data_pcd[0:-1:1,:],
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:10,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
# plot += plt_points_2
plot.display()

Output()

array([[103641.35 , 115533.555,  21979.879],
       [111183.21 , 125663.31 ,  32434.633],
       [104624.836, 145108.67 ,  51137.227],
       ...,
       [ 92224.96 ,  85270.33 ,  37213.715],
       [113269.32 , 122941.75 ,  32622.406],
       [ 80575.76 ,  81486.74 ,  91271.62 ]], dtype=float32)

In [4]:
pbar.update(1)
pbar.set_description('masking data pcd')
obs_mask_file = loadmat(f'{args.dataset_dir}/ObsMask/ObsMask{args.scan}_10.mat')
ObsMask, BB, Res = [obs_mask_file[attr] for attr in ['ObsMask', 'BB', 'Res']]
BB = BB.astype(np.float32)

patch = args.patch_size
patch = 600
inbound = ((data_down >= BB[:1]-patch) & (data_down < BB[1:]+patch*2)).sum(axis=-1) ==3
data_in = data_down[inbound]

data_grid = np.around((data_in - BB[:1]) / Res).astype(np.int32)
grid_inbound = ((data_grid >= 0) & (data_grid < np.expand_dims(ObsMask.shape, 0))).sum(axis=-1) ==3
data_grid_in = data_grid[grid_inbound]
in_obs = ObsMask[data_grid_in[:,0], data_grid_in[:,1], data_grid_in[:,2]].astype(np.bool_)
data_in_obs = data_in[grid_inbound][in_obs]

masking data pcd:  38%|█████████████████████████████████████▌                                                              | 3/8 [01:38<03:10, 38.05s/it]

In [5]:
data_in_obs

array([], shape=(0, 3), dtype=float32)

In [24]:
pbar.update(1)
pbar.set_description('read STL pcd')
stl_pcd = o3d.io.read_point_cloud(f'{args.dataset_dir}/Points/stl/stl{args.scan:03}_total.ply')
stl = np.asarray(stl_pcd.points)

pbar.update(1)
pbar.set_description('compute data2stl')
nn_engine.fit(stl)
dist_d2s, idx_d2s = nn_engine.kneighbors(data_in_obs, n_neighbors=1, return_distance=True)
max_dist = args.max_dist
mean_d2s = dist_d2s[dist_d2s < max_dist].mean()

pbar.update(1)
pbar.set_description('compute stl2data')
ground_plane = loadmat(f'{args.dataset_dir}/ObsMask/Plane{args.scan}.mat')['P']

stl_hom = np.concatenate([stl, np.ones_like(stl[:,:1])], -1)
above = (ground_plane.reshape((1,4)) * stl_hom).sum(-1) > 0
stl_above = stl[above]

nn_engine.fit(data_in)
dist_s2d, idx_s2d = nn_engine.kneighbors(stl_above, n_neighbors=1, return_distance=True)
mean_s2d = dist_s2d[dist_s2d < max_dist].mean()

pbar.update(1)
pbar.set_description('visualize error')
vis_dist = args.visualize_threshold
R = np.array([[1,0,0]], dtype=np.float64)
G = np.array([[0,1,0]], dtype=np.float64)
B = np.array([[0,0,1]], dtype=np.float64)
W = np.array([[1,1,1]], dtype=np.float64)
data_color = np.tile(B, (data_down.shape[0], 1))
data_alpha = dist_d2s.clip(max=vis_dist) / vis_dist
data_color[ np.where(inbound)[0][grid_inbound][in_obs] ] = R * data_alpha + W * (1-data_alpha)
data_color[ np.where(inbound)[0][grid_inbound][in_obs][dist_d2s[:,0] >= max_dist] ] = G
write_vis_pcd(f'{args.pts_dir}/vis_{args.scan:03}_d2s.ply', data_down, data_color)
stl_color = np.tile(B, (stl.shape[0], 1))
stl_alpha = dist_s2d.clip(max=vis_dist) / vis_dist
stl_color[ np.where(above)[0] ] = R * stl_alpha + W * (1-stl_alpha)
stl_color[ np.where(above)[0][dist_s2d[:,0] >= max_dist] ] = G
write_vis_pcd(f'{args.pts_dir}/vis_{args.scan:03}_s2d.ply', stl, stl_color)

pbar.update(1)
pbar.set_description('done')
pbar.close()
over_all = (mean_d2s + mean_s2d) / 2
print(mean_d2s, mean_s2d, over_all)

with open(f'{args.pts_dir}/cf.txt', 'w') as f:
    f.write(f'Mean d2s: {mean_d2s}\n')
    f.write(f'Mean s2d: {mean_s2d}\n')
    f.write(f'Over all: {over_all}\n')




done: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:38<00:00, 19.76s/it]

2.2374554995430644 1.5291523261336084 1.8833039128383364


In [27]:


plt_points = k3d.points(data_in_obs,
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:5,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot += plt_points_2
plot.display()

Output()